In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate
import tensorflow as tf

In [2]:
train_df = pd.read_csv("./C01_train_encoded.csv")
#train_df.info()

In [3]:
train_df.head(10)

,id,project_is_approved,project_grade_category_enc_0,project_grade_category_enc_1,project_grade_category_enc_2,project_grade_category_enc_3,school_state_enc_0,school_state_enc_1,school_state_enc_2,school_state_enc_3,...,description_enc_477,description_enc_478,description_enc_479,description_enc_480,description_enc_481,description_enc_482,description_enc_483,description_enc_484,description_enc_485,description_enc_486
0,p036502,1.0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,p039565,0.0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,p233823,1.0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,p185307,0.0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,p013780,1.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,p063374,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,p103285,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,p181781,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,p114989,1.0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,p191410,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_data = train_df.values
X = train_data[:, 2:]  # id,正解データ列以外を抽出
Y = train_data[:, 1]  # 正解データ列のみ抽出

In [5]:
X.shape

(182080, 1830)

In [6]:
Y.shape

(182080,)

In [7]:
Y[Y[0] == 1].sum() / Y.size

0.8476823374340949

In [8]:
num_classes = 1

# I used LabelEncoder for convert list[0.0, 1.0, 1.0] to list[0, 1, 1]
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=6, shuffle=True)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(1024, input_dim=X.shape[1], activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid')) # softmax/sigmoid

model.compile(optimizer='adam',
#              loss='sparse_categorical_crossentropy',
#              loss='categorical_crossentropy',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Cross-Validation
for train, valid in kfold.split(X, Y):
    model.fit(X[train], Y[train], epochs=1)
    loss, accuracy = model.evaluate(X[valid], Y[valid])
    print("loss = {:.2f}".format(loss))
    print("acc = {:.2f}".format(accuracy))

Epoch 1/1
30348/30348 [==============================] - 3s 113us/step
loss = 0.37
acc = 0.85
Epoch 1/1
30348/30348 [==============================] - 3s 111us/step
loss = 0.37
acc = 0.85
Epoch 1/1
30346/30346 [==============================] - 3s 111us/step
loss = 0.36
acc = 0.85
Epoch 1/1
30346/30346 [==============================] - 3s 113us/step
loss = 0.35
acc = 0.86
Epoch 1/1
30346/30346 [==============================] - 3s 114us/step
loss = 0.33
acc = 0.87
Epoch 1/1
30346/30346 [==============================] - 3s 114us/step
loss = 0.30
acc = 0.88


In [10]:
test_df = pd.read_csv("./C01_test_encoded.csv")
#test_df.info()

In [11]:
test_df.head(5)

,id,project_is_approved,project_grade_category_enc_0,project_grade_category_enc_1,project_grade_category_enc_2,project_grade_category_enc_3,school_state_enc_0,school_state_enc_1,school_state_enc_2,school_state_enc_3,...,description_enc_477,description_enc_478,description_enc_479,description_enc_480,description_enc_481,description_enc_482,description_enc_483,description_enc_484,description_enc_485,description_enc_486
0,p233245,NaN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,p096795,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,p236235,NaN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,p233680,NaN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,p171879,NaN,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
test_data = test_df.values
xs_test = test_data[:, 2:]

In [13]:
y_test = model.predict(xs_test, verbose=1)
#y_test = model.predict_proba(xs_test, batch_size=1000, verbose=1)

78035/78035 [==============================] - 12s 159us/step


In [14]:
y_test.shape

(78035, 1)

In [15]:
y_test[0:20,:]

array([[0.8339554 ],
       [0.96561605],
       [0.9655514 ],
       [0.77860796],
       [0.97649926],
       [0.98337483],
       [0.9077242 ],
       [0.91928107],
       [0.8710192 ],
       [0.9849385 ],
       [0.99167985],
       [0.8791952 ],
       [0.6383276 ],
       [0.955831  ],
       [0.97530013],
       [0.91538197],
       [0.94745946],
       [0.9519366 ],
       [0.78849185],
       [0.3298739 ]], dtype=float32)

In [16]:
y_test[y_test[:,0] >= 0.815].size / y_test.size

0.7801755622477093

In [17]:
y_test[y_test[:,0] < 0.815].size

17154

In [18]:
y_test[y_test[:,0] < 0.815]

array([[0.77860796],
       [0.6383276 ],
       [0.78849185],
       ...,
       [0.55568874],
       [0.7818217 ],
       [0.34356233]], dtype=float32)

In [19]:
Submission = pd.DataFrame()
Submission["id"] = test_df["id"].astype(str)
Submission["project_is_approved"] = y_test.astype(str)

In [20]:
import csv
Submission.to_csv("D01_Submission_10.csv", quoting=csv.QUOTE_NONNUMERIC , quotechar='"', index=False)